In [ ]:
import numpy as np
import pandas as pd
from random import seed
from random import randrange
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix, recall_score
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler,SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from matplotlib import pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
import scikitplot as skplt
dataset = np.load(r'C:\Users\Justin\Desktop\Unizeug\Abgeschlossen\PBL\uniprot_binary_binding_2.5.npz', mmap_mode='r')
labels = np.genfromtxt(fname = r'C:\Users\Justin\Desktop\Unizeug\Abgeschlossen\PBL\binary_binding_2.5(1).txt',dtype='str')


aminoacids = []      

for protein in dataset:
    amino = dataset[str(protein)]
    aminoacids.append([protein, amino])

aminoacids.sort()           #alphabetisch sortieren


In [ ]:
newLabels = [] 

for protein, label in labels:                     #labels zahl zuweisen und multilabels rauskürzen
    if (label == 'small'):
        newLabels.append([protein, 0])
    elif (label == 'metal'):
        newLabels.append([protein, 1])
    elif (label == 'nuc'):
        newLabels.append([protein, 2])          #small = 514,metal = 317, nuc = 86   #+428 nuc & + 197 metal = +625

filteredAmino = []                     #Schnittmenge
filteredLabels = []                    #filteredLabels = newLabels

for amino in aminoacids:
    for label in newLabels:
        if (amino[0] == label[0]):
            filteredAmino.append(amino[1])
            filteredLabels.append(label[1])            #proteinId's raus
            break

 

                                                     #b = [[seqvec1][seqvec2]] 1119 x 1024 und l = 1119 [1,2,...] len(l) = 1119
                                                     #b.shape = (m,n)and  l.shape should be (m)
print(filteredAmino)
print(filteredLabels)
a = (np.array(filteredAmino))
b = (np.array(filteredLabels))
print(a.shape)
print(b.shape)
print(sorted(Counter(b).items()))

In [ ]:
a_train, a_test, b_train, b_test = train_test_split(a, b, random_state = 0, test_size = 0.1)
print(a_train)
print(b_train)
print(b_test)
print(a_test)                      #test absplitten -> training set nehmen zum weiterarbeiten und zum undersampling
print("Dataset sizes:\nWhole set: {}\nTraining Set: {}\nTest Set: {}".
      format(len(b), len(b_train), len(b_test)))
print(sorted(Counter(b_train).items()))

In [ ]:
clf = DummyClassifier(strategy='stratified', random_state=0)        #stratified/most_frequent/uniform
clf.fit(a_test, b_test)

clf.score(a_test, b_test)

In [ ]:
cv = StratifiedKFold(n_splits = 5)
classifier = MLPClassifier(early_stopping = True)

print(format(sum(b_train==1)))
print(format(sum(b_train==0)))
print(format(sum(b_train==2)))

sm = SMOTE(random_state = 0)
a_train_res, b_train_res = sm.fit_sample(a_train, b_train.ravel())
print(format(a_train_res.shape))
print(format(b_train_res.shape))
print(format(sum(b_train_res==1)))
print(format(sum(b_train_res==0)))
print(format(sum(b_train_res==2)))

params ={ 'hidden_layer_sizes': [(30,)]} #testing different parameters #max_iter maybe?

grid = GridSearchCV(estimator = classifier, cv = cv, param_grid = params,  
                    return_train_score=True)
grid.fit(a_train_res, b_train_res)
#grid.fit(a_test,b_test)



cv_results = pd.DataFrame(grid.cv_results_)
print(cv_results)


In [ ]:
best_classifier = grid.best_estimator_
b_pred = best_classifier.predict(a_test)
pred_score = best_classifier.score(a_test, b_test)

# Calculate confusion matrix (showing tp, fp, tn, fn)   
cm = confusion_matrix(b_test, b_pred)
cmn = (cm.T / cm.astype(np.float).sum(axis=1)).T
print(cmn)
print('Acc: {}'.format(round(pred_score, 3)))#TP + TN / n
#precision_score(b_train_res, b_pred, average='macro')


In [ ]:
f1_score(b_test, b_pred, average='macro')

In [ ]:
#training set prediction
print(classification_report(b_train_res, b_pred))

In [ ]:
print(classification_report(b_test,b_pred)) #test set prediction

In [ ]:
#Random prediction on same set as mine

bb_pred = clf.predict(a_train_res)
pred_score = best_classifier.score(a_train_res, b_train_res)
print(classification_report(b_train_res, bb_pred))

In [ ]:
labels = ['small', 'metal', 'nuc']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cmn, cmap=plt.cm.Blues)
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('Expected')
skplt.metrics.plot_confusion_matrix(b_test, b_pred, normalize=True)
plt.show()
plt.show()